In [1]:
from collections import Counter

train = pd.read_csv('data/train.csv', dtype={'Upc': object})
test = pd.read_csv('data/test.csv', dtype={'Upc': object})
train_visits = train.VisitNumber.values
test_visits = test.VisitNumber.values
train = train.drop('TripType', axis=1)

data = pd.concat([train, test])

def check_digit(x):
    try:
        odd = map(int, ','.join(x[-1::-2]).split(','))
        even = map(int, ','.join(x[-2::-2]).split(','))
        sum_odd3 = sum(odd) * 3
        total = sum_odd3 + sum(even)
        rem = total % 10
        if rem == 0:
            return rem
        return 10 - rem
    except:
        return -9999

data['check'] = data.Upc.apply(check_digit)

In [2]:
def full_upc(x):
    try:
        if len(x) < 12:
            missing = 11 - len(x)
            zeros = ['0'] * missing
            xx = zeros + ','.join(x).split(',') + [str(check_digit(x))]
            xx = ''.join(xx)
            return xx
    except:
        return -9999

data['full_upc'] = data.Upc.apply(full_upc)

data = data.drop('Upc', axis=1)

In [3]:
def company(x):
    try:
        p = x[:6]
        if p == '000000':
            return x[-5]
        return p
    except:
        return -9999

data['company'] = data.full_upc.apply(company)

#data.to_csv('data/fulldata.csv', index=False)

In [4]:
from sklearn import preprocessing
data = data.fillna(-9999)
data = data.drop(['DepartmentDescription', 'full_upc', 'company'], axis=1)
for col in data.columns:
    print(col)
    if col not in [u'VisitNumber', u'ScanCount']:
        lbl = preprocessing.LabelEncoder()
        data[col] = lbl.fit_transform(data[col].values)


data.to_csv('data/full_data_enc.csv', index=False)

VisitNumber
Weekday
ScanCount
FinelineNumber
check


In [6]:
data = pd.read_csv('data/full_data_enc.csv', usecols=['VisitNumber', 'ScanCount', 'FinelineNumber'])
visit = np.unique(data.VisitNumber.values)
with open('data/fulldata_fineline_bought.libsvm', 'w') as f:
    for k, v in enumerate(visit):
        trip = data[data['VisitNumber'] == v]
        trip = trip.drop('VisitNumber', axis=1)
        if trip.empty:
            f.write('{}\n'.format(str(-1)))
            continue
        pos_trip = trip[trip['ScanCount'] > 0]
        if pos_trip.empty:
            f.write('{}\n'.format(str(-1)))
            continue
        pos_trip = pos_trip.sort_values(by='FinelineNumber')
        p = np.array(pos_trip.groupby(
            'FinelineNumber').agg(sum).reset_index())
        f.write('{}\n'.format(
            ' '.join([str(-1)] + [str(i[0] + 1) + ':' + str(i[1]) for i in p])))
        if k % 1000 == 0:
            print(k)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
74000
75000
76000
77000
78000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
98000
99000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
151000
152000
154000
155000
156000
157000
158000
159000
160000
161000
162000
163000
164000
165000
166000
167000
1

In [8]:
data = pd.read_csv('data/full_data_enc.csv')
visit = np.unique(data.VisitNumber.values)
with open('data/fulldata_fineline_count.libsvm', 'w') as f:
    for k, v in enumerate(visit):
        trip = data[data['VisitNumber'] == v]
        trip = trip.drop('VisitNumber', axis=1)
        if trip.empty:
            f.write('{}\n'.format(str(-1)))
            continue
        trip = trip.sort_values(by='FinelineNumber')
        p = np.array(trip.groupby(
            'FinelineNumber').agg('count').reset_index())
        f.write('{}\n'.format(
            ' '.join([str(-1)] + [str(i[0] + 1) + ':' + str(i[1]) for i in p])))
        if k % 1000 == 0:
            print(k)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [10]:
data = pd.read_csv('data/fulldata.csv')
visit = np.unique(data.VisitNumber.values)
with open('data/fulldata_fineline_per_trip.libsvm', 'w') as f:
    for i, v in enumerate(visit):
        df = data[data['VisitNumber'] == v]
        if i %100 == 0: print(i)
        x = ','.join(list(df.apply(lambda x:'%s' % (x['FinelineNumber']),axis=1).ravel()))
        f.write('{}\n'.format(x))

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

133000
133100
133200
133300
133400
133500
133600
133700
133800
133900
134000
134100
134200
134300
134400
134500
134600
134700
134800
134900
135000
135100
135200
135300
135400
135500
135600
135700
135800
135900
136000
136100
136200
136300
136400
136500
136600
136700
136800
136900
137000
137100
137200
137300
137400
137500
137600
137700
137800
137900
138000
138100
138200
138300
138400
138500
138600
138700
138800
138900
139000
139100
139200
139300
139400
139500
139600
139700
139800
139900
140000
140100
140200
140300
140400
140500
140600
140700
140800
140900
141000
141100
141200
141300
141400
141500
141600
141700
141800
141900
142000
142100
142200
142300
142400
142500
142600
142700
142800
142900
143000
143100
143200
143300
143400
143500
143600
143700
143800
143900
144000
144100
144200
144300
144400
144500
144600
144700
144800
144900
145000
145100
145200
145300
145400
145500
145600
145700
145800
145900
146000
146100
146200
146300
146400
146500
146600
146700
146800
146900
147000
147100
147200

In [11]:
data = pd.read_csv('data/full_data_enc.csv', usecols=['VisitNumber', 'ScanCount', 'FinelineNumber'])
visit = np.unique(data.VisitNumber.values)
with open('data/fulldata_fineline_return.libsvm', 'w') as f:
    for k, v in enumerate(visit):
        trip = data[data['VisitNumber'] == v]
        trip = trip.drop('VisitNumber', axis=1)
        if trip.empty:
            f.write('{}\n'.format(str(-1)))
            continue
        neg_trip = trip[trip['ScanCount'] < 0]
        if neg_trip.empty:
            f.write('{}\n'.format(str(-1)))
            continue
        neg_trip = neg_trip.sort_values(by='FinelineNumber')
        p = np.array(neg_trip.groupby(
            'FinelineNumber').agg(sum).reset_index())
        f.write('{}\n'.format(
            ' '.join([str(-1)] + [str(i[0] + 1) + ':' + str(i[1]) for i in p])))
        if k % 1000 == 0:
            print(k)

4000
29000
34000
44000
50000
61000
72000
73000
79000
97000
100000
116000
121000
123000
130000
141000
149000
150000
153000


In [ ]:
print(data.shape)
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data['Weekday'].value_counts().plot(kind='bar')

In [ ]:
data['company'].value_counts()

In [ ]:
data.isnull().sum()

In [ ]:
data = pd.read_csv('data/fulldata.csv')
data = data.drop_duplicates('VisitNumber')
visit = np.unique(data.VisitNumber.values)

In [ ]:
WEEKDAY = {'Sunday': 1,
           'Monday': 2,
           'Tuesday': 3,
           'Wednesday': 4,
           'Thursday': 5,
           'Friday': 6,
           'Saturday': 7
           }


with open('data/full_weekday.libsvm', 'w') as f:
    for k, v in enumerate(visit):
        trip = data[data['VisitNumber'] == v]
        w = trip.Weekday.values[0]
        f.write('{}\n'.format(' '.join([str(-1)] + [str(WEEKDAY[w]) + ':' + str(1)])))
        if k % 1000 == 0:
            print(k)